## Part 1: Data processing with pandas

In this section, you will see examples of some commonly used data wrangling tools in Python. In particular, we aim to give you some familiarity with:

* Slicing data frames
* Filtering data
* Grouped counts
* Joining two tables
* NA/Null values


### Practice: Pandas and Wrangling (5%)

This part of the homework is graded manually based on showing the correct outputs after executing each step.

You need to execute each step (run each Cell), in order for the next ones to work.

For the examples that follow, we will be using a toy data set containing information about superheroes in the Arrowverse.  In the `first_seen_on` column, `a` stands for Archer and `f`, Flash.


First, import necessary libraries:

In [80]:
import pandas as pd
import numpy as np

#### Setup

The code below produces the data frames used in the examples:

In [81]:
heroes = pd.DataFrame(
    data={'color': ['red', 'green', 'black',
                    'blue', 'black', 'red'],
          'first_seen_on': ['a', 'a', 'f', 'a', 'a', 'f'],
          'first_season': [2, 1, 2, 3, 3, 1]},
    index=['flash', 'arrow', 'vibe',
           'atom', 'canary', 'firestorm']
)

identities = pd.DataFrame(
    data={'ego': ['barry allen', 'oliver queen', 'cisco ramon',
                  'ray palmer', 'sara lance',
                  'martin stein', 'ronnie raymond'],
          'alter-ego': ['flash', 'arrow', 'vibe', 'atom',
                        'canary', 'firestorm', 'firestorm']}
)

teams = pd.DataFrame(
    data={'team': ['flash', 'arrow', 'flash', 'legends',
                   'flash', 'legends', 'arrow'],
          'hero': ['flash', 'arrow', 'vibe', 'atom',
                   'killer frost', 'firestorm', 'speedy']})

In [82]:
heroes

,color,first_seen_on,first_season
flash,red,a,2
arrow,green,a,1
vibe,black,f,2
atom,blue,a,3
canary,black,a,3
firestorm,red,f,1


In [83]:
identities

,ego,alter-ego
0,barry allen,flash
1,oliver queen,arrow
2,cisco ramon,vibe
3,ray palmer,atom
4,sara lance,canary
5,martin stein,firestorm
6,ronnie raymond,firestorm


In [84]:
teams

,team,hero
0,flash,flash
1,arrow,arrow
2,flash,vibe
3,legends,atom
4,flash,killer frost
5,legends,firestorm
6,arrow,speedy


#### Slice and Dice

##### Column selection by label
To select a column of a `DataFrame` by column label, the safest and fastest way is to use the `.loc` method. General usage looks like `frame.loc[rowname,colname]`. (Reminder that the colon `:` means "everything").  For example, if we want the `color` column of the `heroes` data frame, we would use :

In [85]:
heroes.loc[:,'color']

flash          red
arrow        green
vibe         black
atom          blue
canary       black
firestorm      red
Name: color, dtype: object

The above output can also be got in dataframe format instead of series

In [86]:
heroes.loc[:,['color']]

,color
flash,red
arrow,green
vibe,black
atom,blue
canary,black
firestorm,red


Another way of doing the above is shown below. The below method creates a boolean mask

In [87]:
heroes.loc[:, heroes.columns=='color']

,color
flash,red
arrow,green
vibe,black
atom,blue
canary,black
firestorm,red


Selecting multiple columns is easy. You just need to supply a list of column names. Here we select the color and value columns:

In [88]:
heroes.loc[:, ['color', 'first_season']]

,color,first_season
flash,red,2
arrow,green,1
vibe,black,2
atom,blue,3
canary,black,3
firestorm,red,1


While .loc is invaluable when writing production code, it may be a little too verbose for interactive use. One recommended alternative is the [] method, which takes on the form frame['colname'].

In [89]:
heroes['first_seen_on']

flash        a
arrow        a
vibe         f
atom         a
canary       a
firestorm    f
Name: first_seen_on, dtype: object

##### Row Selection by Label

Similarly, if we want to select a row by its label, we can use the same .loc method.

In [90]:
heroes.loc[['flash', 'vibe'], :]

,color,first_seen_on,first_season
flash,red,a,2
vibe,black,f,2


If we want all the columns returned, we can, for brevity, drop the colon without issue.

In [91]:
heroes.loc[['flash', 'vibe']]

,color,first_seen_on,first_season
flash,red,a,2
vibe,black,f,2


##### General Selection by Label

More generally you can slice across both rows and columns at the same time.  For example:

In [92]:
heroes.loc['flash':'atom', :'first_seen_on']

,color,first_seen_on
flash,red,a
arrow,green,a
vibe,black,f
atom,blue,a


##### Selection by Integer Index

If you want to select rows and columns by position, the Data Frame has an analogous `.iloc` method for integer indexing. Remember that Python indexing starts at 0.

In [93]:
heroes.iloc[:4,:2]

,color,first_seen_on
flash,red,a
arrow,green,a
vibe,black,f
atom,blue,a


#### Filtering with boolean arrays

Filtering is the process of removing unwanted material.  In your quest for cleaner data, you will undoubtedly filter your data at some point: whether it be for clearing up cases with missing values, culling out fishy outliers, or analyzing subgroups of your data set.  For example, we may be interested in characters that debuted in season 3 of Archer.  Note that compound expressions have to be grouped with parentheses.

In [94]:
heroes[(heroes['first_season']==3) & (heroes['first_seen_on']=='a')]

,color,first_seen_on,first_season
atom,blue,a,3
canary,black,a,3


#### Problem Solving Strategy

We want to highlight the strategy for filtering to answer the question above:

* **Identify the variables of interest**
    * Interested in the debut: `first_season` and `first_seen_on`
* **Translate the question into statements one with True/False answers**
    * Did the hero debut on Archer? $\rightarrow$ The hero has `first_seen_on` equal to `a`
    * Did the hero debut in season 3? $\rightarrow$ The hero has `first_season` equal to `3`
* **Translate the statements into boolean statements**
    * The hero has `first_seen_on` equal to `a` $\rightarrow$ `hero['first_seen_on']=='a'`
    * The hero has `first_season` equal to `3` $\rightarrow$ `heroes['first_season']==3`
* **Use the boolean array to filter the data**

Note that compound expressions have to be grouped with parentheses.

For your reference, some commonly used comparison operators are given below.

Symbol | Usage      | Meaning
------ | ---------- | -------------------------------------
==   | a == b   | Does a equal b?
<=   | a <= b   | Is a less than or equal to b?
>=   | a >= b   | Is a greater than or equal to b?
<    | a < b    | Is a less than b?
&#62;    | a &#62; b    | Is a greater than b?
~    | ~p       | Returns negation of p
&#124; | p &#124; q | p OR q
&    | p & q    | p AND q
^  | p ^ q | p XOR q (exclusive or)

An often-used operation missing from the above table is a test-of-membership.  The `Series.isin(values)` method returns a boolean array denoting whether each element of `Series` is in `values`.  We can then use the array to subset our data frame. For example, if we wanted to see which rows of `heroes` had values in $\{1,3\}$, we would use:

In [95]:
heroes[heroes['first_season'].isin([1,3])]

,color,first_seen_on,first_season
arrow,green,a,1
atom,blue,a,3
canary,black,a,3
firestorm,red,f,1


Notice that in both examples above, the expression in the brackets evaluates to a boolean series.  The general strategy for filtering data frames, then, is to write an expression of the form `frame[logical statement]`.

#### Counting Rows

To count the number of instances of a value in a `Series`, we can use the `value_counts` method.  Below we count the number of instances of each color.

In [96]:
heroes['color'].value_counts()

red      2
black    2
green    1
blue     1
Name: color, dtype: int64

A more sophisticated analysis might involve counting the number of instances a tuple appears.  Here we count $(color,value)$ tuples.

In [97]:
heroes.groupby(['color', 'first_season']).size()

color  first_season
black  2               1
       3               1
blue   3               1
green  1               1
red    1               1
       2               1
dtype: int64

This returns a series that has been multi-indexed.  We'll eschew this topic for now.  To get a data frame back, we'll use the `reset_index` method, which also allows us to simulataneously name the new column.

In [98]:
heroes.groupby(['color', 'first_season']).size().reset_index(name='count')

,color,first_season,count
0,black,2,1
1,black,3,1
2,blue,3,1
3,green,1,1
4,red,1,1
5,red,2,1


#### Joining Tables on One Column

Suppose we have another table that classifies superheroes into their respective teams.  Note that `canary` is not in this data set and that `killer frost` and `speedy` are additions that aren't in the original `heroes` set.

For simplicity of the example, we'll convert the index of the `heroes` data frame into an explicit column called `hero`.  A careful examination of the [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) will reveal that joining on a mixture of the index and columns is possible.

In [99]:
heroes['hero'] = heroes.index
heroes

,color,first_seen_on,first_season,hero
flash,red,a,2,flash
arrow,green,a,1,arrow
vibe,black,f,2,vibe
atom,blue,a,3,atom
canary,black,a,3,canary
firestorm,red,f,1,firestorm


##### Inner Join

The inner join below returns rows representing the heroes that appear in both data frames.

In [100]:
pd.merge(heroes, teams, how='inner', on='hero')

,color,first_seen_on,first_season,hero,team
0,red,a,2,flash,flash
1,green,a,1,arrow,arrow
2,black,f,2,vibe,flash
3,blue,a,3,atom,legends
4,red,f,1,firestorm,legends


##### Left and right join
The left join returns rows representing heroes in the `heroes` ("left") data frame, augmented by information found in the `teams` data frame.  Its counterpart, the right join, would return heroes in the `teams` data frame.  Note that the `team` for hero `canary` is an `NaN` value, representing missing data.

In [101]:
pd.merge(heroes, teams, how='left', on='hero')

,color,first_seen_on,first_season,hero,team
0,red,a,2,flash,flash
1,green,a,1,arrow,arrow
2,black,f,2,vibe,flash
3,blue,a,3,atom,legends
4,black,a,3,canary,NaN
5,red,f,1,firestorm,legends


##### Outer join

An outer join on `hero` will return all heroes found in both the left and right data frames.  Any missing values are filled in with `NaN`.

In [102]:
pd.merge(heroes, teams, how='outer', on='hero')

,color,first_seen_on,first_season,hero,team
0,red,a,2.0,flash,flash
1,green,a,1.0,arrow,arrow
2,black,f,2.0,vibe,flash
3,blue,a,3.0,atom,legends
4,black,a,3.0,canary,NaN
5,red,f,1.0,firestorm,legends
6,NaN,NaN,NaN,killer frost,flash
7,NaN,NaN,NaN,speedy,arrow


##### More than one match?

If the values in the columns to be matched don't uniquely identify a row, then a cartesian product is formed in the merge.  For example, notice that `firestorm` has two different egos, so information from `heroes` had to be duplicated in the merge, once for each ego.

In [103]:
pd.merge(heroes, identities, how='inner',
         left_on='hero', right_on='alter-ego')

,color,first_seen_on,first_season,hero,ego,alter-ego
0,red,a,2,flash,barry allen,flash
1,green,a,1,arrow,oliver queen,arrow
2,black,f,2,vibe,cisco ramon,vibe
3,blue,a,3,atom,ray palmer,atom
4,black,a,3,canary,sara lance,canary
5,red,f,1,firestorm,martin stein,firestorm
6,red,f,1,firestorm,ronnie raymond,firestorm


#### Missing Values

There are a multitude of reasons why a data set might have missing values.  The current implementation of Pandas uses the numpy NaN to represent these null values (older implementations even used `-inf` and `inf`).  Future versions of Pandas might implement a true `null` value---keep your eyes peeled for this in updates!  More information can be found [http://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html](http://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html)

Because of the specialness of missing values, they merit their own set of tools.  Here, we will focus on detection.  For replacement, see the docs.

In [104]:
x = np.nan
y = pd.merge(heroes, teams, how='outer', on='hero')['first_season']
y

0    2.0
1    1.0
2    2.0
3    3.0
4    3.0
5    1.0
6    NaN
7    NaN
Name: first_season, dtype: float64

To check if a value is null, we use the `isnull()` method for series and data frames.  Alternatively, there is a `pd.isnull()` function as well.

In [105]:
x.isnull() # won't work since x is neither a series nor a data frame

AttributeError: 'float' object has no attribute 'isnull'

In [106]:
pd.isnull(x)

True

In [107]:
y.isnull()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
7     True
Name: first_season, dtype: bool

In [108]:
pd.isnull(y)

0    False
1    False
2    False
3    False
4    False
5    False
6     True
7     True
Name: first_season, dtype: bool

Since filtering out missing data is such a common operation, Pandas also has conveniently included the analogous `notnull()` methods and function for improved human readability.

In [109]:
y.notnull()

0     True
1     True
2     True
3     True
4     True
5     True
6    False
7    False
Name: first_season, dtype: bool

In [110]:
y[y.notnull()]

0    2.0
1    1.0
2    2.0
3    3.0
4    3.0
5    1.0
Name: first_season, dtype: float64

### Questions (45%)

The practice problems below use the department of transportation's "On-Time" flight data for all flights originating from SFO or OAK in January 2016. Information about the airports and airlines are contained in the comma-delimited files `airports.dat` and `airlines.dat`, respectively.  Both were sourced from http://openflights.org/data.html.

Disclaimer: There is a more direct way of dealing with time data that is not presented in these problems.  This activity is merely an academic exercise.

#### Setup
Run the below cells to set up the tables flights and airports

In [111]:
flights = pd.read_csv("flights.dat", dtype={'sched_dep_time': 'f8', 'sched_arr_time': 'f8'})
# show the first few rows, by default 5
flights.head(3)

,year,month,day,date,carrier,tailnum,flight,origin,destination,sched_dep_time,actual_dep_time,sched_arr_time,actual_arr_time
0,2016,1,1,2016-01-01,AA,N3FLAA,208,SFO,MIA,630.0,628.0,1458.0,1431.0
1,2016,1,2,2016-01-02,AA,N3APAA,208,SFO,MIA,600.0,553.0,1428.0,1401.0
2,2016,1,3,2016-01-03,AA,N3DNAA,208,SFO,MIA,630.0,626.0,1458.0,1431.0


In [112]:
flights.describe()

,year,month,day,flight,sched_dep_time,actual_dep_time,sched_arr_time,actual_arr_time
count,16861.0,16861.0,16861.000000,16861.000000,16861.000000,16399.000000,16861.000000,16365.000000
mean,2016.0,1.0,15.781033,2100.205978,1333.387818,1334.757851,1518.619773,1509.387779
std,0.0,0.0,8.978231,2036.303273,503.163806,518.007117,529.272153,548.227990
min,2016.0,1.0,1.000000,1.000000,5.000000,2.000000,2.000000,1.000000
25%,2016.0,1.0,8.000000,506.000000,915.000000,915.000000,1150.000000,1141.000000
50%,2016.0,1.0,16.000000,1372.000000,1300.000000,1309.000000,1600.000000,1558.000000
75%,2016.0,1.0,24.000000,2771.000000,1725.000000,1736.000000,1945.000000,1948.000000
max,2016.0,1.0,31.000000,6845.000000,2359.000000,2400.000000,2359.000000,2400.000000


In [113]:
airports_cols = [
    'openflights_id',
    'name',
    'city',
    'country',
    'iata',
    'icao',
    'latitude',
    'longitude',
    'altitude',
    'tz',
    'dst',
    'tz_olson',
    'type',
    'airport_dsource'
]

airports = pd.read_csv("airports.dat", names=airports_cols)
airports.head(3)

,openflights_id,name,city,country,iata,icao,latitude,longitude,altitude,tz,dst,tz_olson,type,airport_dsource
0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10.0,U,Pacific/Port_Moresby,NaN,NaN
1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.788700,20,10.0,U,Pacific/Port_Moresby,NaN,NaN
2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10.0,U,Pacific/Port_Moresby,NaN,NaN


#### Question 1.1 (15%)
Lets practice some pandas commands in this sections.

(5%) In the 'flights' table, display unique values present in columns - 'carrier', 'origin', 'destination'

In [114]:
#Code - write one line code to display unique values in column 'carrier'
# write here
out = flights['carrier'].unique()


# testing the code < do not change code below>
print(len(out))
display(out)

11


array(['AA', 'AS', 'B6', 'DL', 'F9', 'HA', 'NK', 'OO', 'UA', 'VX', 'WN'],
      dtype=object)

In [115]:
#Code - write one line code to display unique values in column 'carrier'
# write here
out = flights['origin'].unique()


# testing the code < do not change code below>
print(len(out))
display(out)

2


array(['SFO', 'OAK'], dtype=object)

In [116]:
#Code - write one line code to display unique values in column 'carrier'
# write here
out = flights['destination'].unique()


# testing the code <do not change code below>
print(len(out))
display(out)

79


array(['MIA', 'LAX', 'JFK', 'DFW', 'ORD', 'PHL', 'PHX', 'CLT', 'KOA',
       'SEA', 'PDX', 'PSP', 'SLC', 'OGG', 'HNL', 'LIH', 'BOS', 'FLL',
       'LGB', 'LAS', 'DTW', 'ATL', 'MSP', 'IAH', 'DEN', 'EUG', 'BOI',
       'BUR', 'SMX', 'SAN', 'ACV', 'ASE', 'MRY', 'STL', 'SBA', 'SMF',
       'JAC', 'SAT', 'XNA', 'AUS', 'MTJ', 'MFR', 'MCI', 'MMH', 'OKC',
       'ONT', 'FAT', 'PSC', 'RDM', 'SBP', 'RNO', 'BFL', 'BZN', 'TUS',
       'SNA', 'HDN', 'SUN', 'ABQ', 'EWR', 'MCO', 'MSY', 'IAD', 'IND',
       'DCA', 'RDU', 'CLE', 'RDD', 'OTH', 'BWI', 'PIT', 'ANC', 'DAL',
       'MDW', 'MKE', 'BNA', 'CMH', 'GEG', 'HOU', 'CVG'], dtype=object)

(5%) Display list of all flights with "SFO" as origin and "JFK" as destination run by the carrier "DL" . Only display columns - "origin", "destination" "carrier" and "sched_dep_time"

In [117]:
# write your code here

#out = flights[['origin','destination','sched_dep_time']]

out = flights.loc[(flights['origin'] == 'SFO') & (flights['destination'] == 'JFK') & (flights['carrier'] == 'DL'),['origin','destination','carrier','sched_dep_time']]


# display
out
#display output like this to get full credit (179 rows):
#     origin	destination	carrier	sched_dep_time
# 2473 SFO     JFK          DL      2145.0
# 2478 SFO      JFK         DL      615.0
# 2479 SFO      JFK         DL      1415.0

,origin,destination,carrier,sched_dep_time
2473,SFO,JFK,DL,2145.0
2478,SFO,JFK,DL,615.0
2479,SFO,JFK,DL,1415.0
2480,SFO,JFK,DL,800.0
2502,SFO,JFK,DL,1415.0
...,...,...,...,...
16783,SFO,JFK,DL,1600.0
16787,SFO,JFK,DL,2148.0
16791,SFO,JFK,DL,2148.0
16812,SFO,JFK,DL,625.0


(5%) - AGGREGATION - Find the total number of flights run by each carrier. Sort the list in descending order of number of flights. Just display numeric index,  carrier and number of flights. Hint: check documentationf for: groupby , sort_values, reset_index

Note: Store the group in "out" variable .

In [118]:
def display_carrier_counts(flights):
    #DOUBLE CHECK TO MAKE SURE IT WORKS CORRECTLY
    
    #group them first 
    #reset index 
    #lastly sort by descending
    flightCounts = flights.groupby('carrier').size().reset_index(name = 'count')
    out = flightCounts.sort_values(by = 'count',ascending = False).reset_index(drop = True)
  
    
  ##======= Do not write below this === # return out should remain for full credit
    return out

# Display in below format for full credit
#       carrier   count
#  0    WN        4169
#  1    UA        3687

out = display_carrier_counts(flights)
out

,carrier,count
0,WN,4169
1,UA,3687
2,OO,3325
3,VX,1548
4,AA,1338
5,DL,1073
6,AS,615
7,B6,610
8,F9,186
9,NK,186


In [119]:
# Test code - RUN this code to get full credit
# test code: Note that the sum of values should be equal to the lenght of flights table
out["count"].sum()
# 16861

16861

#### Question 1.2 **(10%)**
It looks like the departure and arrival in `flights` were read in as floating-point numbers.  Write two functions, `extract_hour` and `extract_mins` that converts military time to hours and minutes, respectively. Hint: You may want to use modular arithmetic and integer division. Keep in mind that the data has not been cleaned and you need to check whether the extracted values are valid. Replace all the invalid values with `NaN`. The documentation for `pandas.Series.where` provided [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.where.html) should be helpful.

In [120]:
def extract_hour(time):
    """
    Extracts hour information from military time.

    Args:
        time (float64): series of time given in military format.
          Takes on values in 0.0-2359.0 due to float64 representation.

    Returns:
        array (float64): series of input dimension with hour information.
          Should only take on integer values in 0-23
    """
    # [YOUR CODE HERE]
    
    #cleans data
    time = time.where((time <=2359.0) & (time >= 0) , other=np.nan)
    
    
    # performs integer division isolating the time
    hour = time // 100

    return hour

# test your code to receive credit
test_ser = pd.Series([1030.0, 1259.0, np.nan, 2475], dtype='float64')
extract_hour(test_ser)

# 0    10.0
# 1    12.0
# 2     NaN
# 3     NaN
# dtype: float64

0    10.0
1    12.0
2     NaN
3     NaN
dtype: float64

In [121]:
def extract_mins(time):
    """
    Extracts minute information from military time

    Args:
        time (float64): series of time given in military format.
          Takes on values in 0.0-2359.0 due to float64 representation.

    Returns:
        array (float64): series of input dimension with minute information.
          Should only take on integer values in 0-59
    """
    # [YOUR CODE HERE]
    
    
    
    #cleans data
    time = time.where((time <=2359.0) & (time >= 0) , other=np.nan)
    
    #EXTRACT THE MINUTES 
    mins = time % 100
    return mins


# test your code to receive credit
test_ser = pd.Series([1030.0, 1259.0, np.nan, 2475], dtype='float64')
extract_mins(test_ser)

# 0    30.0
# 1    59.0
# 2     NaN
# 3     NaN
# dtype: float64

0    30.0
1    59.0
2     NaN
3     NaN
dtype: float64

#### Question 1.3 **(15%)**

Using your two functions above, filter the `flights` data for flights that departed 20 or more minutes later than scheduled by comparing `sched_dep_time` and `actual_dep_time`.  You need not worry about flights that were delayed to the next day for this question.

In [122]:
def convert_to_minofday(time):
    """
    Converts military time to minute of day

    Args:
        time (float64): series of time given in military format.
          Takes on values in 0.0-2359.0 due to float64 representation.

    Returns:
        array (float64): series of input dimension with minute of day

    Example: 1:03pm is converted to 783.0
    """
    
#     extract_hour()
#     extract_mins()
    
    # [YOUR CODE HERE]
    # hours * 60 + minutes 
    hours = extract_hour(time)
    minutes = extract_mins(time)
    
    minuteDay = (hours * 60 ) + minutes
    
    return minuteDay

# Test your code  to receive credit
ser = pd.Series([1303, 1200, 2400], dtype='float64')
convert_to_minofday(ser)

# 0    783.0
# 1    720.0
# 2      NaN
# dtype: float64

0    783.0
1    720.0
2      NaN
dtype: float64

In [123]:
def calc_time_diff(x, y):
    """
    Calculates delay times y - x

    Args:
        x (float64): series of scheduled time given in military format.
          Takes on values in 0.0-2359.0 due to float64 representation.
        y (float64): series of same dimensions giving actual time

    Returns:
        array (float64): series of input dimension with delay time
    """
    
    # [YOUR CODE HERE]
    return convert_to_minofday(y) - convert_to_minofday(x)
#     

#Test your code  to receive credit
sched = pd.Series([1303, 1210], dtype='float64')
actual = pd.Series([1304, 1215], dtype='float64')
calc_time_diff(sched, actual)

# 0    1.0
# 1    5.0
# dtype: float64

0    1.0
1    5.0
dtype: float64

In [124]:
flights.head(3)

,year,month,day,date,carrier,tailnum,flight,origin,destination,sched_dep_time,actual_dep_time,sched_arr_time,actual_arr_time
0,2016,1,1,2016-01-01,AA,N3FLAA,208,SFO,MIA,630.0,628.0,1458.0,1431.0
1,2016,1,2,2016-01-02,AA,N3APAA,208,SFO,MIA,600.0,553.0,1428.0,1401.0
2,2016,1,3,2016-01-03,AA,N3DNAA,208,SFO,MIA,630.0,626.0,1458.0,1431.0


In [125]:
### Apply your functions here to calculate delay between `sched_dep_time` and `actual_dep_time` on flights, to receive credit.
# [YOUR CODE HERE]


#ask the TAs if you need to worry about negative values


sched_dep_time = flights['sched_dep_time']
actual_dep_time = flights['actual_dep_time']

# SHOWS ALL DELAY TIMES GREATER THAN 0
delay_V2 = calc_time_diff(actual_dep_time,sched_dep_time) 
delay = delay_V2[delay_V2 > 0] 

# shows just the time 
# delay =  # Series object showing delay time
delay

0        2.0
1        7.0
2        4.0
3        4.0
4        8.0
        ... 
16848    2.0
16849    8.0
16850    5.0
16851    6.0
16852    7.0
Length: 8795, dtype: float64

In [126]:
#flights delayed more than 20mins
# Code here

sched_dep_time = flights['sched_dep_time']
actual_dep_time = flights['actual_dep_time']

# SHOWS ALL DELAY TIMES GREATER THAN 0
delay_V2 = calc_time_diff(actual_dep_time,sched_dep_time) 
delay = delay_V2[delay_V2 > 20]  

delayed20 = delay
delayed20

151      1291.0
153      1224.0
154      1347.0
158       658.0
159      1282.0
          ...  
16646    1350.0
16663    1390.0
16668    1212.0
16681    1336.0
16789    1382.0
Length: 114, dtype: float64

#### Question 1.4 **(5%)**

Using your answer from question 1.2, find the full name of every destination city with a flight from SFO or OAK that was delayed by 60 or more minutes.  The airport codes used in `flights` are IATA codes.  Sort the cities alphabetically. Make sure you remove duplicates. (You may find `drop_duplicates` and `sort_values` helpful.)

In [127]:
# Complete code here to receive credit.
# HINT: You will need to use `delayed` and `airport` dataframes
# [YOUR CODE HERE]



# Filter flights departing from SFO or OAK
sfo_oak_flights = flights[(flights['origin'] == 'SFO') | (flights['origin'] == 'OAK')]

# Calculate the delay for each flight using calc_time_diff
sfo_oak_flights['delay'] = calc_time_diff(sfo_oak_flights['actual_dep_time'], sfo_oak_flights['sched_dep_time'])

# Filter flights with a delay of 60 or more minutes
delayed_flights = sfo_oak_flights[sfo_oak_flights['delay'] >= 60]

# Merge with the 'airports' DataFrame to get city names
delayed_with_city = pd.merge(delayed_flights, airports, left_on='destination', right_on='iata', how='inner')

# Merge with the 'airplanes' DataFrame to get airplane information
delayed_with_airplane = pd.merge(delayed_with_city, airports, left_on='origin', right_on='iata', how='inner')

# Extract the unique destination cities
unique_destination_cities = delayed_with_city['city'].unique()

# Sort the cities alphabetically
delayed_destinations = sorted(unique_destination_cities)


# delayed_destinations_iata = 'blabla'
# delayed_airports = 'blabla' # Dataframe showing airports that satisfy above conditions
# delayed_destinations = 'blabla' # Unique and sorted destination cities
delayed_destinations

['Arcata CA',
 'Atlanta',
 'Baltimore',
 'Boston',
 'Chicago',
 'Cincinnati',
 'Detroit',
 'Eugene',
 'Fort Lauderdale',
 'Fresno',
 'Houston',
 'Las Vegas',
 'Long Beach',
 'Los Angeles',
 'Medford',
 'Miami',
 'New Orleans',
 'New York',
 'Newark',
 'Ontario',
 'Palm Springs',
 'Portland',
 'Raleigh-durham',
 'Redding',
 'Redmond-Bend',
 'Sacramento',
 'Salt Lake City',
 'San Diego',
 'Santa Barbara',
 'Seattle',
 'Tucson',
 'Washington']

## Part 2: Web scraping and data collection

### Note and Setup

Here, you will practice collecting and processing data in Python. By the end of this exercise hopefully you should look at the wonderful world wide web without fear, comforted by the fact that anything you can see with your human eyes, a computer can see with its computer eyes. In particular, we aim to give you some familiarity with:

* Using HTTP to fetch the content of a website
* HTTP Requests (and lifecycle)
* RESTful APIs
    * Authentication (OAuth)
    * Pagination
    * Rate limiting
* JSON vs. HTML (and how to parse each)
* HTML traversal (CSS selectors)

Since everyone loves food (presumably), the ultimate end goal of this homework will be to acquire the data to answer some questions and hypotheses about the restaurant scene in Chicago (which we will get to later). We will download __both__ the metadata on restaurants in Chicago from the Yelp API and with this metadata, retrieve the comments/reviews and ratings from users on restaurants.

**Library Documentation:**

For solving this part, you need to look up online documentation for the Python packages you will use:

* Standard Library:
    * [io](https://docs.python.org/3/library/io.html)
    * [time](https://docs.python.org/3/library/time.html)
    * [json](https://docs.python.org/3/library/json.html)

* Third Party
    * [requests](https://requests.readthedocs.io/en/latest/)
    * [Beautiful Soup (version 4)](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
    * [yelp-fusion](https://www.yelp.com/developers/documentation/v3/get_started)

**Note:** You may come across a `yelp-python` library online. The library is deprecated and incompatible with the current Yelp API, so do not use the library.

First, import necessary libraries:

In [128]:
import io, time, json
import requests
from bs4 import BeautifulSoup

### Authentication and working with APIs



There are various authentication schemes that APIs use, listed here in relative order of complexity:

* No authentication
* [HTTP basic authentication](https://en.wikipedia.org/wiki/Basic_access_authentication)
* Cookie based user login
* OAuth (v1.0 & v2.0, see this [post](http://stackoverflow.com/questions/4113934/how-is-oauth-2-different-from-oauth-1) explaining the differences)
* API keys
* Custom Authentication

For the NYT example below (**Q2.1**), since it is a publicly visible page we did not need to authenticate. HTTP basic authentication isn't too common for consumer sites/applications that have the concept of user accounts (like Facebook, LinkedIn, Twitter, etc.) but is simple to setup quickly and you often encounter it on with individual password protected pages/sites.

Cookie based user login is what the majority of services use when you login with a browser (i.e. username and password). Once you sign in to a service like Facebook, the response stores a cookie in your browser to remember that you have logged in (HTTP is stateless). Each subsequent request to the same domain (i.e. any page on `facebook.com`) also sends the cookie that contains the authentication information to remind Facebook's servers that you have already logged in.

Many REST APIs however use OAuth (authentication using tokens) which can be thought of a programmatic way to "login" _another_ user. Using tokens, a user (or application) only needs to send the login credentials once in the initial authentication and as a response from the server gets a special signed token. This signed token is then sent in future requests to the server (in place of the user credentials).

A similar concept common used by many APIs is to assign API Keys to each client that needs access to server resources. The client must then pass the API Key along with _every_ request it makes to the API to authenticate. This is because the server is typically relatively stateless and does not maintain a session between subsequent calls from the same client. Most APIs (including Yelp) allow you to pass the API Key via a special HTTP Header: `Authorization: Bearer <API_KEY>`. Check out the [docs](https://www.yelp.com/developers/documentation/v3/authentication) for more information.

### Question 2.1: Basic HTTP Requests w/o authentication **(5%)**

First, let's do the "hello world" of making web requests with Python to get a sense for how to programmatically access web pages: an (unauthenticated) HTTP GET to download a web page.

Fill in the funtion to use `requests` to download and return the raw HTML content of the URL passed in as an argument. As an example try the following NYT article (on Youtube's algorithmic recommendation): [https://www.nytimes.com/2019/03/29/technology/youtube-online-extremism.html](https://www.nytimes.com/2019/03/29/technology/youtube-online-extremism.html)

Your function should return a tuple of: (`<status_code>`, `<text>`). (Hint: look at the **Library documentation** listed earlier to see how `requests` should work.)

In [129]:
def retrieve_html(url):
    """
    Return the raw HTML at the specified URL.

    Args:
        url (string):

    Returns:
        status_code (integer):
        raw_html (string): the raw HTML content of the response, properly encoded according to the HTTP headers.
    """

    # [YOUR CODE HERE]
    r = requests.get(url)
    status_code = r.status_code
    text = r.text

    return status_code, text

In [130]:
#  test your function here to receive credit
test = retrieve_html('https://martinheinz.dev/blog/31')
print(test)
# (200, '<!DOCTYPE html>\n<html > )

(200, '<!DOCTYPE html>\n<html >\n<head><meta charset="utf-8">\n<title>Scraping News and Articles From Public APIs with Python | Martin Heinz | Personal Website & Blog</title>\n<meta name="viewport" content="width=device-width, initial-scale=1">\n<meta name="twitter:title" content="Scraping News and Articles From Public APIs with Python">\n<meta name="twitter:text:title" content="Scraping News and Articles From Public APIs with Python">\n<meta name="og:url" content="https://martinheinz.dev/blog/31">\n<meta name="og:type" content="article">\n<meta name="article:published_time" content="2020-08-20T17:30:00Z">\n<meta name="article:section" content="Technology">\n<meta name="twitter:description" content="<p>\nWhether you are data scientist, programmer or AI specialist, you surely can put huge number of news articles to some good use. Getting those articles c...">\n<meta name="og:description" content="<p>\nWhether you are data scientist, programmer or AI specialist, you surely can put huge n

Now while this example might have been fun, we haven't yet done anything more than we could with a web browser. To really see the power of programmatically making web requests we will need to interact with an API. For the rest of this lab we will be working with the [Yelp API](https://www.yelp.com/developers/documentation/v3/get_started) and Yelp data (for an extensive data dump see their [Academic Dataset Challenge](https://www.yelp.com/dataset_challenge)).

### Yelp API Access

The reasons for using the Yelp API are 3 fold:

1. Incredibly rich dataset that combines:
    * entity data (users and businesses)
    * preferences (i.e. ratings)
    * geographic data (business location and check-ins)
    * temporal data
    * text in the form of reviews
    * and even images.
2. Well [documented API](https://www.yelp.com/developers/documentation/v3/get_started) with thorough examples.
3. Extensive data coverage so that you can find data that you know personally (from your home town/city or account). This will help with understanding and interpreting your results.

Yelp used to use OAuth tokens but has now switched to API Keys. **For the sake of backwards compatibility Yelp still provides a Client ID and Secret for OAuth, but you will not need those for this assignment.**

To access the Yelp API, we will need to go through a few more steps than we did with the first NYT example. Most large web scale companies use a combination of authentication and rate limiting to control access to their data to ensure that everyone using it abides. The first step (even before we make any request) is to setup a Yelp account if you do not have one and get API credentials.

1. Create a [Yelp](https://www.yelp.com/login) account (if you do not have one already)
2. [Generate API keys](https://www.yelp.com/developers/v3/manage_app) (if you haven't already). You will only need the API Key (not the Client ID or Client Secret) -- more on that later.

Now that we have our accounts setup we can start making requests!


### Question 2.2: Authenticated HTTP Request with the Yelp API **(15%)**



First, store your Yelp credentials in a local file (kept out of version control) which you can read in to authenticate with the API. This file can be any format/structure since you will fill in the function stub below.

For example, you may want to store your key in a file called `yelp_api_key.txt` (run in terminal):
```bash
echo 'YOUR_YELP_API_KEY' > yelp_api_key.txt
```
if you are using google colab: Run this in a cell:
```bash
!echo 'YOUR_YELP_API_KEY' > yelp_api_key.txt
```

**KEEP THE API KEY FILE PRIVATE AND OUT OF VERSION CONTROL (and definitely do not submit them to Gradescope!)**

You can then read from the file using:

In [131]:
with open('yelp_api_key.txt', 'r') as f:
    api_key = f.read().replace('\n','')
#     print(api_key)
    # verify your api_key is correct
# DO NOT FORGET TO CLEAR THE OUTPUT TO KEEP YOUR API KEY PRIVATE

In [132]:
def read_api_key(filepath):
    """
    Read the Yelp API Key from file.

    Args:
        filepath (string): File containing API Key
    Returns:
        api_key (string): The API Key
    """

    # feel free to modify this function if you are storing the API Key differently
    with open(filepath, 'r') as f:
        return f.read().replace('\n','')

Using the Yelp API, fill in the following function stub to make an authenticated request to the [search](https://www.yelp.com/developers/documentation/v3/business_search) endpoint. Remember Yelp allows you to pass the API Key via a special HTTP Header: `Authorization: Bearer <API_KEY>`. Check out the [docs](https://www.yelp.com/developers/documentation/v3/authentication) for more information.

In [133]:
def location_search_params(api_key, location, **kwargs):
    """
    Construct url, headers and url_params. Reference API docs (link above) to use the arguments
    """
    # [YOUR CODE HERE]
    # What is the url endpoint for search?
    url = 'https://api.yelp.com/v3/businesses/search'
    # How is Authentication performed?
    headers = {
    
        'Authorization':f'Bearer {api_key}'
    
    }
    # SPACES in url is problematic. How should you handle location containing spaces?
    url_params = {
        
        'location' : location,
        
    }
    # Include keyword arguments in url_params

    url_params.update(kwargs)
    
    
    return url, headers, url_params


Hint: `**kwargs` represent keyword arguments that are passed to the function. For example, if you called the function `location_search_params(api_key, location, offset=0, limit=50)`. The arguments `api_key` and `location` are called *positional arguments* and key-value pair arguments are called **keyword arguments**. Your `kwargs` variable will be a python dictionary with those keyword arguments.

In [134]:
# Test your code here to receive credit.
api_key = "test_api_key_xyz"
location = "Chicago"
url, headers, url_params = location_search_params(api_key, location, offset=0, limit=50)
url, headers, url_params
# ('https://<hidden_url_check_search_endpoint_docs_to_get_answer>',
#  {'Authorization': 'Bearer test_api_key_xyz'},
#  {'location': 'Chicago', 'offset': 0, 'limit': 50})

('https://api.yelp.com/v3/businesses/search',
 {'Authorization': 'Bearer test_api_key_xyz'},
 {'location': 'Chicago', 'offset': 0, 'limit': 50})

Now use `location_search_params(api_key, location, **kwargs)` to actually search restaurants from Yelp API. Most of the code is provided to you. Complete the `api_get_request` function given below.

In [135]:
def api_get_request(url, headers, url_params):
    """
    Send a HTTP GET request and return a json response

    Args:
        url (string): API endpoint url
        headers (dict): A python dictionary containing HTTP headers including Authentication to be sent
        url_params (dict): The parameters (required and optional) supported by endpoint

    Returns:
        results (json): response as json
    """
    http_method = 'GET'
    response = requests.get(url,headers = headers,params = url_params)
    return response.json()


def yelp_search(api_key, location, offset=0):
    """
    Make an authenticated request to the Yelp API.

    Args:
        api_key (string): Your Yelp API Key for Authentication
        location (string): Business Location
        offset (int): param for pagination

    Returns:
        total (integer): total number of businesses on Yelp corresponding to the location
        businesses (list): list of dicts representing each business
    """
    url, headers, url_params = location_search_params(api_key, location, offset=0)
    response_json = api_get_request(url, headers, url_params)
    return response_json["total"], list(response_json["businesses"])

# test your code here to receive credit
# [YOUR CODE HERE]
with open('yelp_api_key.txt', 'r') as f:
    api_key = f.read().replace('\n','')

num_records, data = yelp_search(api_key, 'Chicago')
print(num_records)
#9000
print(len(data))
#20
print(list(map(lambda x: x['name'], data)))
#['Girl & The Goat', 'Wildberry Pancakes and Cafe', 'Au Cheval', 'The Purple Pig', "Lou Malnati's Pizzeria", 'Art Institute of Chicago', "Bavette's Bar & Boeuf", 'Cafe Ba-Ba-Reeba!', 'Smoque BBQ', 'Little Goat Diner', "Pequod's Pizzeria", 'Alinea', 'Quartino Ristorante', "Kuma's Corner - Belmont", "Joe's Seafood, Prime Steak & Stone Crab", 'Crisp', "Portillo's Hot Dogs", 'Sapori Trattoria', 'Xoco', "Molly's Cupcakes"]

9000
20
['Girl & The Goat', 'The Purple Pig', 'Cafe Ba-Ba-Reeba!', 'Au Cheval', "Bavette's Bar & Boeuf", 'S.K.Y.', 'Aba', 'The Dearborn', 'Sapori Trattoria', 'Alinea', 'Ema', "Joe's Seafood, Prime Steak & Stone Crab", 'Little Bad Wolf', 'Boka', 'etta - Bucktown', 'KAI ZAN', "Cindy's Rooftop", 'Monteverde', 'Prime & Provisions', 'The Gage']


### Parameterization and Pagination

Now that we have completed the "hello world" of working with the Yelp API, we are ready to really fly! The rest of the exercise will have a bit less direction since there are a variety of ways to retrieve the requested information but you should have all the component knowledge at this point to work with the API. Yelp being a fairly general platform actually has many more business than just restaurants, but by using the flexibility of the API we can ask it to only return the restaurants.



And before we can get any reviews on restaurants, we need to actually get the metadata on ALL of the restaurants in Chicago. Notice above that while Yelp told us that there are ~240, the response contained fewer actual `Business` objects. This is due to pagination and is a safeguard against returning __TOO__ much data in a single request (what would happen if there were 100,000 restaurants?) and can be used in conjuction with _rate limiting_ as well as a way to throttle and protect access to Yelp data.

> As a thought exercise, consider: If an API has 1,000,000 records, but only returns 10 records per page and limits you to 5 requests per second... how long will it take to acquire ALL of the records contained in the API?

One of the ways that APIs are an improvement over plain web scraping is the ability to make __parameterized__ requests. Just like the Python functions you have been writing have arguments (or parameters) that allow you to customize its behavior/actions (an output) without having to rewrite the function entirely, we can parameterize the queries we make to the Yelp API to filter the results it returns.

### Question 2.3: Acquire all of the restaurants in Chicago on Yelp **(10%)**



Again using the [API documentation](https://www.yelp.com/developers/documentation/v3/business_search) for the `search` endpoint, fill in the following function to retrieve all of the _Restuarants_ (using categories) for a given query. Again you should use your `read_api_key()` function outside of the `all_restaurants()` stub to read the API Key used for the requests. You will need to account for __pagination__ and __[rate limiting](https://www.yelp.com/developers/faq)__ to:

1. Retrieve all of the Business objects (# of business objects should equal `total` in the response). **Paginate by querying 10 restaurants each request.**
2. Pause slightly (at least 200 milliseconds) between subsequent requests so as to not overwhelm the API (and get blocked).  

As always with API access, make sure you follow all of the [API's policies](https://www.yelp.com/developers/api_terms) and use the API responsibly and respectfully.

**DO NOT MAKE TOO MANY REQUESTS TOO QUICKLY OR YOUR KEY MAY BE BLOCKED**

In [136]:
import math

def paginated_restaurant_search_requests(api_key, location, total):
    """
    Returns a list of tuples (url, headers, url_params) for paginated search of all restaurants
    Args:
        api_key (string): Your Yelp API Key for Authentication
        location (string): Business Location
        total (int): Total number of items to be fetched
    Returns:
        results (list): list of tuple (url, headers, url_params)
    """
    # HINT: Use total, offset and limit for pagination
    # You can reuse function location_search_params(...)
    # [YOUR CODE HERE]
    
    offset = 0
    limit = 10
    numLimit = math.ceil(total/limit)
    cat = 'restaraunts'
    paginatedResults = []
  
    
    for x in range(numLimit):
        parameters = {
        'offset': offset,
        'limit': limit,
        'categories': cat,
        }
    
        url,head,url_param = location_search_params(api_key,location,**parameters)
        paginatedResults.append((url,head,url_param))
        offset += limit
        
  

    # returns url, headers, url_params
    return paginatedResults


# Test your code here to receive credit.

# with open('yelp_api_key.txt', 'r') as f:
#     api_key = f.read().replace('\n','')
api_key = "test_api_key_xyz"#replace this
location = "Chicago"
all_restaurants_requests = paginated_restaurant_search_requests(api_key, location, 15)
all_restaurants_requests

# [('https:<hidden>',
#   {'Authorization': 'Bearer test_api_key_xyz'},
#   {'location': 'Chicago',
#    'offset': 0,
#    'limit': 10,
#    'categories': '<hidden>'}),
#  ('https:<hidden>',
#   {'Authorization': 'Bearer test_api_key_xyz'},
#   {'location': 'Chicago',
#    'offset': 10,
#    'limit': 10,
#    'categories': '<hidden>'})]

[('https://api.yelp.com/v3/businesses/search',
  {'Authorization': 'Bearer test_api_key_xyz'},
  {'location': 'Chicago', 'offset': 0, 'limit': 10}),
 ('https://api.yelp.com/v3/businesses/search',
  {'Authorization': 'Bearer test_api_key_xyz'},
  {'location': 'Chicago', 'offset': 10, 'limit': 10})]

In [137]:
from time import sleep
def all_restaurants(api_key, location):
    """
    Construct the pagination requests for ALL the restaurants on Yelp for a given location.

    Args:
        api_key (string): Your Yelp API Key for Authentication
        location (string): Business Location

    Returns:
        results (list): list of dicts representing each restaurant
    """
    # What keyword arguments should you pass to get first page of restaurants in Yelp
    url, headers, url_params = location_search_params(api_key, location, offset=0, limit=10)
    #
    response_json = api_get_request(url, headers, url_params)
    total_items = response_json["total"]

    all_restaurants_request = paginated_restaurant_search_requests(api_key, location, total_items)

    # Use returned list of (url, headers, url_params) and function api_get_request to retrive all restaurants
    # REMEMBER to pause slightly after each request.
    # [YOUR CODE HERE]
    #     api_get_request returns a JSON file 
    
    allRestaurants = []
    
    for x in all_restaurants_request:
        url,headers,params = x
        response_json = api_get_request(url, headers, params)
        restairants = response_json.get("businesses",[])
        allRestaurants.extend(restairants)
        #one second sleep just to be safe
        sleep(.500)
    return allRestaurants

You can test your function with an individual neighborhood in Chicago (for example, Greektown). Chicago itself has a lot of restaurants... meaning it will take a lot of time to download them all.

In [138]:
api_key = read_api_key('yelp_api_key.txt')
data = all_restaurants(api_key, 'Greektown, Chicago, IL')
print(len(data))
# 92
print(list(map(lambda x:x['name'], data)))
# ['Greek Islands Restaurant', 'Artopolis', 'Meli Cafe & Juice Bar', 'Athena Greek Restaurant', 'WJ Noodles', ...]

128
['Greek Islands Restaurant', 'Monteverde', 'Trivoli Tavern', 'Athena Greek Restaurant', 'Green Street Smoked Meats', 'CityBird', 'Artopolis', 'Sepia', "Formento's", '9 Muses', 'Viaggio Ristorante & Lounge', 'El Che Steakhouse & Bar', 'Tamashii Ramen', 'High Five Ramen', 'Rye Deli & Drink', 'The Allis', 'Meli Cafe & Juice Bar', 'Green Street Local', 'Zeus Restaurant', 'Sawada Coffee', 'Booze Box', 'Spectrum Bar and Grill', 'Primos Chicago Pizza', 'Xi’an Cuisine', 'Suenos x Soho House', 'Dawali Jerusalem Kitchen', "Nando's Peri-Peri", "Philly's Best", 'Mr Greek Gyros', 'Omakase Yume', 'Ciao! Cafe & Wine Lounge', "Nancy's Pizza West Loop", 'Parlor Pizza Bar', 'TenGoku Aburiya', 'Lola’s Restaurant & Bar', 'Jubilee Juice & Grill', "Giordano's", 'J.P. Graziano Grocery', 'H Mart - Chicago', 'Blaze Pizza', 'Slightly Toasted', 'Stelios Bottles & Bites', 'Fox Bar', "Dugan's", "Nonna's Pizza & Sandwiches", 'La Colombe Coffee', 'Taco Lulú', 'Taco Burrito King - Greektown', 'SGD Dubu So Gong Do

Now that we have the metadata on all of the restaurants in Greektown (or at least the ones listed on Yelp), we can retrieve the reviews and ratings. The Yelp API gives us aggregate information on ratings but it doesn't give us the review text or individual users' ratings for a restaurant. For that we need to turn to web scraping, but to find out what pages to scrape we first need to parse our JSON from the API to extract the URLs of the restaurants.

In general, it is a best practice to separate the act of __downloading__ data and __parsing__ data. This ensures that your data processing pipeline is modular and extensible (and autogradable ;). This decoupling also solves the problem of expensive downloading but cheap parsing (in terms of computation and time).


### Question 2.4: Parse the API Responses and Extract the URLs **(5%)**




Because we want to separate the __downloading__ from the __parsing__, fill in the following function to parse the URLs pointing to the restaurants on `yelp.com`. As input your function should expect a string of [properly formatted JSON](http://www.json.org/) (which is similar to __BUT__ not the same as a Python dictionary) and as output should return a Python list of strings. Hint: print your `data` to see the JSON-formatted information you have. The input JSON will be structured as follows :

```json
{
  "total": 8228,
  "businesses": [
    {
      "rating": 4,
      "price": "$",
      "phone": "+14152520800",
      "id": "four-barrel-coffee-san-francisco",
      "is_closed": false,
      "categories": [
        {
          "alias": "coffee",
          "title": "Coffee & Tea"
        }
      ],
      "review_count": 1738,
      "name": "Four Barrel Coffee",
      "url": "https://www.yelp.com/biz/four-barrel-coffee-san-francisco",
      "coordinates": {
        "latitude": 37.7670169511878,
        "longitude": -122.42184275
      },
      "image_url": "http://s3-media2.fl.yelpcdn.com/bphoto/MmgtASP3l_t4tPCL1iAsCg/o.jpg",
      "location": {
        "city": "San Francisco",
        "country": "US",
        "address2": "",
        "address3": "",
        "state": "CA",
        "address1": "375 Valencia St",
        "zip_code": "94103"
      },
      "distance": 1604.23,
      "transactions": ["pickup", "delivery"]
    }
  ],
  "region": {
    "center": {
      "latitude": 37.767413217936834,
      "longitude": -122.42820739746094
    }
  }
}
```

In [139]:
def parse_api_response(data):
    """
    Parse Yelp API results to extract restaurant URLs.

    Args:
        data (string): String of properly formatted JSON.

    Returns:
        (list): list of URLs as strings from the input JSON.
    """

#     # [YOUR CODE HERE]

    urlList = []
    
    urlData = data["businesses"]
    urlList.append([x["url"] for x in urlData])

#     # Printing the extracted URLs
#     for url in urls:
#         print("URL:", url)
    
    
    return urlList


# test your code here to receive credit
url, headers, url_params = location_search_params(api_key, "Bridgeport, Chicago, IL", offset=0)
response_text = api_get_request(url, headers, url_params)
parse_api_response(response_text)
# ['https://www.yelp.com/biz/nana-chicago?adjust_creative=ioqEYAcUhZO272qCIvxcVA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=ioqEYAcUhZO272qCIvxcVA',
#  'https://www.yelp.com/biz/bridgeport-coffee-chicago-4?adjust_creative=ioqEYAcUhZO272qCIvxcVA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=ioqEYAcUhZO272qCIvxcVA',
# ...]


[['https://www.yelp.com/biz/the-duck-inn-chicago?adjust_creative=jDUfPTzxc7JGj_CVQD9oSw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=jDUfPTzxc7JGj_CVQD9oSw',
  'https://www.yelp.com/biz/francos-ristorante-chicago?adjust_creative=jDUfPTzxc7JGj_CVQD9oSw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=jDUfPTzxc7JGj_CVQD9oSw',
  'https://www.yelp.com/biz/kimski-chicago?adjust_creative=jDUfPTzxc7JGj_CVQD9oSw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=jDUfPTzxc7JGj_CVQD9oSw',
  'https://www.yelp.com/biz/mins-noodle-house-%E6%B8%94%E5%AE%B6%E9%87%8D%E5%BA%86%E5%B0%8F%E9%9D%A2-chicago-32?adjust_creative=jDUfPTzxc7JGj_CVQD9oSw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=jDUfPTzxc7JGj_CVQD9oSw',
  'https://www.yelp.com/biz/shinya-ramen-house-chicago-3?adjust_creative=jDUfPTzxc7JGj_CVQD9oSw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=jDUfPTzxc7JGj_CVQD9oSw',
  'https://www

As we can see, JSON is quite trivial to parse (which is not the case with HTML as we will see in a second) and work with programmatically. This is why it is one of the most ubiquitous data serialization formats (especially for ReSTful APIs) and a huge benefit of working with a well defined API if one exists. But APIs do not always exists or provide the data we might need, and as a last resort we can always scrape web pages...

### Working with Web Pages (and HTML)

Think of APIs as similar to accessing an application's database itself (something you can interactively query and receive structured data back). But the results are usually in a somewhat raw form with no formatting or visual representation (like the results from a database query). This is a benefit _AND_ a drawback depending on the end use case. For data science and _programatic_ analysis this raw form is quite ideal, but for an end user requesting information from a _graphical interface_ (like a web browser) this is very far from ideal since it takes some cognitive overhead to interpret the raw information. And vice versa, if we have HTML it is quite easy for a human to visually interpret it, but to try to perform some type of programmatic analysis we first need to parse the HTML into a more structured form.

> As a general rule of thumb, if the data you need can be accessed or retrieved in a structured form (either from a bulk download or API) prefer that first. But if the data you want (and need) is not as in our case we need to resort to alternative (messier) means.

Going back to the "hello world" example of question 2.1 with the NYT, we will do something similar to retrieve the HTML of the Yelp site itself (rather than going through the API programmatically) as text.
> However, we will use saved HTML pages to reduce excessive traffic to the Yelp website.

### Question 2.5: Parse a Yelp restaurant Page **(10%)**

Using `BeautifulSoup`, parse the HTML of a single Yelp restaurant page to extract the reviews in a structured form as well as the URL to the next page of reviews (or `None` if it is the last page). Fill in following function stubs to parse a single page of reviews and return:
* the reviews as a structured Python dictionary
* the HTML element containing the link/url for the next page of reviews (or None).

For each review be sure to structure your Python dictionary as follows (to be graded correctly). The order of the keys doesn't matter, only the keys and the data type of the values:

```python
{
    'author': str
    'rating': float
    'date': str ('yyyy-mm-dd')
    'description': str
}

{
    'author': 'Topsy Kretts'
    'rating': 4.7
    'date': '2016-01-23'
    'description': "Wonderful!"
}
```

There can be issues with Beautiful Soup using various parsers, for maximum compatibility (and fewest errors) initialize the library with the default (and Python standard library parser): `BeautifulSoup(markup, "html.parser")`.

Most of the function has been provided to you:

In [140]:
url_lookup = {
"https://www.yelp.com/biz/the-jibarito-stop-chicago-2?start=225":"parse_page_test1.html",
"https://www.yelp.com/biz/the-jibarito-stop-chicago-2?start=245":"parse_page_test2.html"
}

def html_fetcher(url):
    """
    Return the raw HTML at the specified URL.
    Args:
        url (string):

    Returns:
        status_code (integer):
        raw_html (string): the raw HTML content of the response, properly encoded according to the HTTP headers.
    """
    html_file = url_lookup.get(url)
    with open(html_file, 'rb') as file:
        html_text = file.read()
        return 200, html_text


def parse_page(html):
    """
    Parse the reviews on a single page of a restaurant.

    Args:
        html (string): String of HTML corresponding to a Yelp restaurant

    Returns:
        tuple(list, string): a tuple of two elements
            first element: list of dictionaries corresponding to the extracted review information
            second element: URL for the next page of reviews (or None if it is the last page)
    """
    soup = BeautifulSoup(html,'html.parser')
    url_next = soup.find('link',rel='next')
    if url_next:
        url_next = url_next.get('href')
    else:
        url_next = None

    reviews = soup.find_all('div', itemprop="review")
    reviews_list = []
    # HINT: print reviews to see what http tag to extract
    # [YOUR CODE HERE]
    
    for review in reviews:
        reviewDict = {
            'author': str(review.find('meta', itemprop="author")["content"]),
            'rating': float(review.find('meta', itemprop="ratingValue")["content"]),
            'date': review.find('meta', itemprop="datePublished")["content"],
            'description': str(review.find('p', itemprop="description").get_text(strip=True))
        }
        reviews_list.append(reviewDict)

    return reviews_list, url_next

# Test your implementation here to receive credit.
code, html = html_fetcher("https://www.yelp.com/biz/the-jibarito-stop-chicago-2?start=225")
reviews_list, url_next = parse_page(html)
print(len(reviews_list)) # 20
print(url_next) #https://www.yelp.com/biz/the-jibarito-stop-chicago-2?start=245

20
https://www.yelp.com/biz/the-jibarito-stop-chicago-2?start=245


### Question 2.6: Extract all Yelp reviews for a Single Restaurant ****

So now that we have parsed a single page, and figured out a method to go from one page to the next we are ready to combine these two techniques and actually crawl through web pages!

Using the provided `html_fetcher` (for a real use-case you would use `requests`), programmatically retrieve __ALL__ of the reviews for a __single__ restaurant (provided as a parameter). Just like the API was paginated, the HTML paginates its reviews (it would be a very long web page to show 300 reviews on a single page) and to get all the reviews you will need to parse and traverse the HTML. As input your function will receive a URL corresponding to a Yelp restaurant. As output return a list of dictionaries (structured the same as question 2.5) containing the relevant information from the reviews. You can use `parse_page()` here.

In [141]:
def extract_reviews(url, html_fetcher):
    """
    Retrieve ALL of the reviews for a single restaurant on Yelp.

    Parameters:
        url (string): Yelp URL corresponding to the restaurant of interest.
        html_fetcher (function): A function that takes url and returns html status code and content

    Returns:
        reviews (list): list of dictionaries containing extracted review information
    """
    reviews = []
    # [YOUR CODE HERE]
    # HINT: Use function `parse_page(html)` multiple times until no next page exists
    #while loop since we dont know how many pages we gotta do
    while url:
        errorCode,html = html_fetcher(url)
        if code != 200:
            return reviews
        
        review,url = parse_page(html)
        reviews.extend(review)

    return reviews

You can test your function with this code:

In [142]:
# test your function here to receive credit.
data = extract_reviews('https://www.yelp.com/biz/the-jibarito-stop-chicago-2?start=225', html_fetcher=html_fetcher)
print(len(data))
# 35
print(data[0])
# {'author': 'Jason S.', 'rating': 5.0, 'date': '2016-05-02', 'description': "This was one of my favorite food trucks ..."}


35
{'author': 'Jason S.', 'rating': 5.0, 'date': '2016-05-02', 'description': "This was one of my favorite food trucks but as of last fall they've opened a brick and mortar restaurant in the Pilsen neighborhood...the perfect success story of how a person can start out with a food truck and grow their business into a restaurant. The food is always delicious and the service is great!"}
